In [ ]:
# whether to run/make the kdeplot with marginal histograms gif
# it's very expensive/time consuming
make_gif = True
# whether to do the other expensive seaborn/kde stuff
do_kde_stuff = True

In [ ]:
import os.path
import numpy as np
from numpy import linalg as la
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animate
import matplotlib.gridspec as gridspec
import gif
# for vscode, use 'ipympl' for animations and 'inline', otherwise
# for browser, use 'notebook'
# %matplotlib ipympl
%matplotlib inline
# this is for colorbars on subplots
from mpl_toolkits.axes_grid1 import make_axes_locatable
# for rendering latex
from IPython.display import display, Math, Latex
import matplotlib.font_manager
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"]})

fname = "../data/particles"
start_fname = fname + "1.txt"
f_ens = "../data/ens.txt"

In [ ]:
with open(f_ens) as f:
    N_ens = int(f.readline())
    
with open(start_fname) as f:
    shapeData = f.readline()
    dim = int(f.readline())
    p = f.readline()

omega = np.zeros(2);

params = p.split()
IC_type_space = float(params[0])
IC_type_mass = float(params[1])
omega[0]= float(params[2])
omega[1]= float(params[3])
X0_space = float(params[4])
hat_pct = float(params[5])
X0_mass = float(params[6])
maxT = float(params[7])
dt = float(params[8])
D = float(params[9])
pctRW = float(params[10])
cdist_coeff = float(params[11])
cutdist = float(params[12])

In [ ]:
# delta IC
sigma = 0.0

shapeData = shapeData.split()
shapeData = [int(i) for i in shapeData]
Np = shapeData[0]
Nsteps = shapeData[1] + 1

In [ ]:
X = np.ndarray([dim, Np, Nsteps])
tmpX = np.ndarray([Np * Nsteps])
tmpY = np.ndarray([Np * Nsteps])
tmpXY = np.ndarray([dim, Np, Nsteps])

for e in range(1, N_ens + 1):
    fname_ens = "../data/particles" + str(e) + ".txt"
    with open(fname_ens) as f:
        data = np.loadtxt(f, skiprows=3)
        tmpX = np.reshape((data[:, 0]), [Np, Nsteps], 'f')
        tmpY = np.reshape((data[:, 1]), [Np, Nsteps], 'f')
        tmpXY = np.stack((tmpX, tmpY), axis=0)
        if e == 1:
            X = tmpXY
        else:
            X = np.append(X, tmpXY, axis=1)

In [ ]:
L = omega[1] - omega[0]

def analytic1d(X, t, sigma, D, L):
    sol =  (1 / np.sqrt(2 * np.pi * (sigma + 2 * D * t)))\
        * np.exp(-((0.5 * L - X[:])**2 / (2 * (sigma + 2 * D * t))));
    return sol

def analytic2d(dim, X, Y, t, sigma, D, L):
    sol = (1 / (2 * np.pi * (sigma + 2 * D * t)))\
          * np.exp(-(((X0_space - X)**2 + (X0_space - Y)**2) / (2 * (sigma + 2 * D * t))));
    return sol

In [ ]:
# number of histogram bins
nBins = int(np.floor(np.sqrt(Np) / 2))
if nBins % 2 == 0:
    nBins += 1

In [ ]:
# plot the initial condition
fig = plt.figure()
plt.hist2d(X[0, :, 0], X[1, :, 0], density=True, bins=nBins, cmap='cividis')
plt.title('Initial Histogram')
plt.show()

In [ ]:
xplot = np.linspace(min(X[0, :, -1]), max(X[0, :, -1]), 100)
yplot = np.linspace(min(X[1, :, -1]), max(X[1, :, -1]), 100)
xx, yy = np.meshgrid(xplot, yplot)
asoln_surf = analytic2d(2, xx, yy, maxT, sigma, D, L)
asoln = analytic2d(2, X[0, :, -1], X[1, :, -1], maxT, sigma, D, L)

particle_mean = [np.mean(X[0, :, -1]), np.mean(X[1, :, -1])]
particle_std = [np.std(X[0, :, -1]), np.std(X[1, :, -1])]

print(np.max(np.max(asoln)))
print(f'sample mean = [{particle_mean[0]}, {particle_mean[1]}]')
print(f'sample std. dev. [sigma_x, sigma_y] =  [{particle_std[0]}, {particle_std[1]}]')
print('analytic mean = {}'.format(X0_space))
print('true sigma_[x,y] = sqrt(2 D maxT) = {}'.format(np.sqrt(2.0 * D * maxT)))

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 18))

plt.axes(axs[0, 0])
plt.title('Final PT Histogram')
plt.hist2d(X[0, :, -1], X[1, :, -1], density=True,
                                     bins=nBins, cmap='cividis', label='PT')
plt.colorbar()
plt.axes(axs[0, 1])
plt.title('Analytic Solution Histogram')
plt.hist2d(X[0, :, -1], X[1, :, -1], density=True, weights=asoln,
                                     bins=nBins, cmap='cividis')
plt.colorbar()
plt.axes(axs[1, 0])
plt.title('Analytic Solution at Particles')
plt.scatter(X[0, :, -1], X[1, :, -1], c=asoln, cmap='cividis', label='Analytic at Particles')
plt.colorbar()
plt.axes(axs[1, 1])
plt.title('Analytic Solution on Meshgrid')
plt.scatter(xx, yy, c=asoln_surf, cmap='cividis')
plt.colorbar()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
plt.axes(axs[0])
plt.title('Final PT Histogram')
plt.hist2d(X[0, :, -1], X[1, :, -1], density=True,
                                     bins=nBins, cmap='cividis', label='PT')
plt.colorbar()
plt.axes(axs[1])
plt.title('Analytic Solution Histogram')
plt.hist2d(X[0, :, -1], X[1, :, -1], density=True, weights=asoln,
                                     bins=nBins, cmap='cividis')
plt.colorbar()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
ax = plt.axes(axs[0])
plt.title('PT KDE Plot Contours')
sns.kdeplot(x=X[0, :, -1], y=X[1, :, -1], fill=True, cmap='cividis', levels=10,
            cut=8.0e-4, cbar=True)
plt.axes(axs[1])
asoln_mask = asoln_surf
plt.title('Contours of Analytic Solution on Meshgrid')
import matplotlib.ticker as tkr
loctr = tkr.MaxNLocator(prune='lower')
plt.contourf(xx, yy, asoln_mask, cmap='cividis', levels=10, locator=loctr)
plt.colorbar()

In [ ]:
def analytic2d_fit(X, Y, t, init_sigma, sigma_x, sigma_y, D, mean_x, mean_y):
    sol = (1 / (2 * np.pi * (sigma + 2 * np.sqrt(sigma_x * sigma_y) * t)))\
          * np.exp(-(((mean_x - X)**2 / (2 * (sigma + 2 * sigma_x * t)) +
                      (mean_y - Y)**2 / (2 * (sigma + 2 * sigma_y * t)))))
    return sol


def analytic_fit(_X, sigma_x, sigma_y, mean_x, mean_y):
    XXX, YYY = np.meshgrid(_X[0, :], _X[1, :])
    sol = analytic2d_fit(XXX, YYY, maxT, 0, sigma_x, sigma_y, D, mean_x, mean_y)
    return np.ravel(sol)

from scipy.optimize import curve_fit

# Produce 2D histogram
nBins = 15
H, xedges, yedges = np.histogram2d(X[0, :, -1], X[1, :, -1], nBins, density=True)
H = np.ravel(H)
bin_centers_x = (xedges[:-1] + xedges[1 :]) / 2.0
bin_centers_y = (yedges[:-1] + yedges[1 :]) / 2.0

Xin = np.vstack([bin_centers_x, bin_centers_y])

an_stdDev = np.sqrt(2.0 * D * maxT)
# Curve Fit parameters--Note that coeff = [sigma_x, sigma_y, mean_x, mean_y]
coeff, var_matrix = curve_fit(analytic_fit, Xin, H, p0=[an_stdDev, an_stdDev, 25, 25])

In [ ]:
def analytic_fit_unravel(_x, _y, sigma_x, sigma_y, mean_x, mean_y):
    sol = analytic2d_fit(_x, _y, maxT, 0, sigma_x, sigma_y, D, mean_x, mean_y)
    return sol

xx, yy = np.meshgrid(xplot, yplot)

fit_ans_pts = analytic2d_fit(X[0, :, -1], X[1, :, -1], maxT, 0, coeff[0], coeff[1], D, coeff[2], coeff[3])
fit_ans_mgrid = analytic_fit_unravel(xx, yy, coeff[0], coeff[1], coeff[2], coeff[3])
asoln_surf = analytic2d(2, xx, yy, maxT, sigma, D, L)
abs_particle_error = np.abs(fit_ans_mgrid - asoln_surf)

fig, axs = plt.subplots(3, 2, figsize=(12, 18))
plt.axes(axs[0, 0])
plt.title('Fitted PDF at Particles')
plt.scatter(X[0, :, -1], X[1, :, -1], c=fit_ans_pts, cmap='cividis')
plt.colorbar()
plt.axes(axs[1, 0])
plt.title('Analytic Solution at Particles')
plt.scatter(X[0, :, -1], X[1, :, -1], c=asoln, cmap='cividis', label='Analytic at Particles')
plt.colorbar()
plt.axes(axs[2, 0])
plt.title('Absolute Difference')
plt.scatter(X[0, :, -1], X[1, :, -1], c=np.abs(asoln - fit_ans_pts), cmap='cividis', label='Analytic at Particles')
plt.colorbar()
plt.axes(axs[0, 1])
plt.title('Fitted PDF on Meshgrid')
plt.scatter(xx, yy, c=fit_ans_mgrid, cmap='cividis')
plt.colorbar()
plt.axes(axs[1, 1])
plt.title('Analytic Solution on Meshgrid')
plt.scatter(xx, yy, c=asoln_surf, cmap='cividis', label='Analytic on Meshgrid')
plt.colorbar()
plt.axes(axs[2, 1])
plt.title('Absolute Difference')
plt.scatter(xx, yy, c=abs_particle_error, cmap='cividis', label='Analytic at Particles')
plt.colorbar()
plt.show()

In [ ]:
print('2-norm error = {}'.format(la.norm(abs_particle_error)))
print('inf-norm error = {}'.format(la.norm(abs_particle_error, np.inf)))
print('max analytic (particles, meshgrid) = {}, {}'.format(np.amax(asoln), np.amax(np.amax(asoln_surf))))
print('max fitted (particles, meshgrid) = {}, {}'.format(np.amax(fit_ans_pts), np.amax(np.amax(fit_ans_mgrid))))
print('fitted: [mu_x, mu_y], [sigma_x, sigma_y] = [{}, {}], [{}, {}]'.format(coeff[3],
                                                                             coeff[2],
                                                                             coeff[1],
                                                                             coeff[0]))
print('particle: [mu_x, mu_y], [sigma_x, sigma_y]' +
      '= [{}, {}], [{}, {}]'.format(particle_mean[0], particle_mean[1],
      particle_std[0], particle_std[1]))
print('analytic: [mu_x, mu_y], [sigma_x, sigma_y]' +
      '= [{}, {}], [{}, {}]'.format(X0_space, X0_space,
      np.sqrt(2.0 * D * maxT), np.sqrt(2.0 * D * maxT)))
print('true sigma_[x,y] = [sqrt(2 D maxT)] = {}\n'.format(np.sqrt(2.0 * D * maxT)))

print('MSE = {}'.format(np.mean(abs_particle_error**2)))
print('abs difference btw. max of analytical, fitted = {}'.format(np.abs(np.amax(asoln) - np.amax(fit_ans_pts))))
print('abs difference in mean_[x,y], std_[x,y] btw. analytical, fitted =' + 
      '[{}, {}], [{}, {}]'.format(np.abs(coeff[3] - X0_space),
                                  np.abs(coeff[2] - X0_space), 
                                  np.abs(coeff[1] - np.sqrt(2.0 * D * maxT)),
                                  np.abs(coeff[0] - np.sqrt(2.0 * D * maxT))))

#### Below here is the fancypants Seaborn/kdeplot stuff, including generating a gif that is very cool but takes **forever**

In [ ]:
if make_gif or do_kde_stuff:    
    class SeabornFig2Grid():

        def __init__(self, seaborngrid, fig,  subplot_spec):
            self.fig = fig
            self.sg = seaborngrid
            self.subplot = subplot_spec
            if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
                isinstance(self.sg, sns.axisgrid.PairGrid):
                self._movegrid()
            elif isinstance(self.sg, sns.axisgrid.JointGrid):
                self._movejointgrid()
            self._finalize()

        def _movegrid(self):
            """ Move PairGrid or Facetgrid """
            self._resize()
            n = self.sg.axes.shape[0]
            m = self.sg.axes.shape[1]
            self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot)
            for i in range(n):
                for j in range(m):
                    self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

        def _movejointgrid(self):
            """ Move Jointgrid """
            h= self.sg.ax_joint.get_position().height
            h2= self.sg.ax_marg_x.get_position().height
            r = int(np.round(h/h2))
            self._resize()
            self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot)

            self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1])
            self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1])
            self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])

        def _moveaxes(self, ax, gs):
            #https://stackoverflow.com/a/46906599/4124317
            ax.remove()
            ax.figure=self.fig
            self.fig.axes.append(ax)
            self.fig.add_axes(ax)
            ax._subplotspec = gs
            ax.set_position(gs.get_position(self.fig))
            ax.set_subplotspec(gs)

        def _finalize(self):
            plt.close(self.sg.fig)
            self.fig.canvas.mpl_connect("resize_event", self._resize)
            self.fig.canvas.draw()

        def _resize(self, evt=None):
            self.sg.fig.set_size_inches(self.fig.get_size_inches())

In [ ]:
if do_kde_stuff:
    kdf = pd.DataFrame(np.transpose(X[:, :, -1]), columns=['X', 'Y'])

    g0 = sns.JointGrid(x="X", y="Y", data=kdf, xlim=[0, 50], ylim=[0, 50])
    g0.plot_joint(sns.scatterplot)

    g1 = sns.JointGrid(x="X", y="Y", data=kdf, xlim=[0, 50], ylim=[0, 50])
    g1.plot_joint(sns.kdeplot, thresh=1e-3, bw_adjust=1.4, levels=10, fill=True)
    g1.plot_marginals(sns.histplot, kde=True)

    fig = plt.figure(figsize=(13,8))
    gs = gridspec.GridSpec(1, 2)

    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])

    gs.tight_layout(fig)

In [ ]:
if do_kde_stuff:
    f, ax = plt.subplots(figsize=(7, 7)) 
    sns.kdeplot(data = kdf, fill=True, ax=ax, x='X', y='Y', thresh=1e-3,
                bw_adjust=1.4, levels=10)
    ax.set_xlim([0, 50])
    ax.set_ylim([0, 50])

In [ ]:
if do_kde_stuff:
    sns.jointplot(data = kdf, x='X', y='Y', kind='kde', thresh=1e-3, bw_adjust=1.4, levels=10, fill=True)

In [ ]:
if do_kde_stuff:
    g = sns.jointplot(data = kdf, x='X', y='Y')
    g.plot_joint(sns.kdeplot, thresh=1e-3, bw_adjust=1.4, levels=10, fill=True)
    ax.set_xlim([0, 50])
    ax.set_ylim([0, 50])

In [ ]:
if make_gif:
    images = []

    @gif.frame
    def plotter(df):
        g0 = sns.JointGrid(x="X", y="Y", data=df, xlim=[0, 50], ylim=[0, 50], space=0)
        g0.plot_joint(sns.scatterplot)

        g1 = sns.JointGrid(x="X", y="Y", data=df, xlim=[0, 50], ylim=[0, 50], space=0)
        g1.plot_joint(sns.kdeplot, thresh=1e-3, bw_adjust=1.4, levels=10, fill=True,
                    warn_singular=False)
        g1.plot_marginals(sns.histplot, kde=True)

        fig = plt.figure(figsize=(13,8))
        gs = gridspec.GridSpec(1, 2)

        mg0 = SeabornFig2Grid(g0, fig, gs[0])
        mg1 = SeabornFig2Grid(g1, fig, gs[1])

        gs.tight_layout(fig)

    for frame in range(0, Nsteps, 10):
        df = pd.DataFrame(np.transpose(X[:, :, frame]), columns = ['X', 'Y'])
        images.append(plotter(df))
    gif.save(images, '2d_rw_kdeJointPlot.gif', duration=500)